In [1]:
import pandas as pd
import pymysql as mc 
import os
import sys
import time
import numpy as np
from sklearn.linear_model import LogisticRegression
import getpass
from sklearn.utils import shuffle
from multiprocessing import Pool

In [2]:
pw = getpass.getpass()

········


In [3]:
cnx = mc.connect(user='root',password=pw,
                 host='localhost',db='mlb_practicum',port=3306)

In [14]:
query = """SELECT Pitch3.pitcherID, Pitch3.gameID, abNum, AVG(runs_created) AS runs_created,AVG(gameDate) AS gameDate 
            FROM Pitch3 INNER JOIN reliever_stuff rs ON 
            (rs.gameID = Pitch3.gameID AND rs.pitcherID = Pitch3.pitcherID)
            GROUP BY Pitch3.gameID, Pitch3.pitcherID,abNum"""
df = pd.read_sql_query(query,cnx)

In [16]:
df = df.sort_values(by=['gameDate','gameID','abNum'])

In [17]:
df.head()

,pitcherID,gameID,abNum,runs_created,gameDate
49665,408072,ANA201004210,55,-0.222,20100421.0
49666,408072,ANA201004210,56,-0.150,20100421.0
49667,408072,ANA201004210,57,-0.098,20100421.0
49668,425530,ANA201004210,67,-0.205,20100421.0
367276,460283,OAK201004210,44,-0.222,20100421.0


In [11]:
query2 = """SELECT gameID, pitcherID FROM reliever_stuff"""
pitcher_game_df = pd.read_sql_query(query2,cnx)

In [24]:
pitcher_game_df.iloc[20000]

gameID       CLE201008130
pitcherID          501925
Name: 20000, dtype: object

In [33]:
get_stats(('CLE201008130','501925'))

88


['501925', 'CLE201008130', 0, 0, 0, 0, 0]

In [21]:
def convert_game_to_date_int(gameID):
    if len(gameID)==12:
        year = gameID[3:7]
        month = gameID[7:9]
        day = gameID[9:11]
        return int(year+month+day)
    elif len(gameID)==26:
        year = gameID[:4]
        month = gameID[5:7]
        day = gameID[8:10]
        return int(year+month+day)

In [34]:
def get_stats(tup):
    gameID = tup[0]
    playerID = tup[1]
    gameDate = convert_game_to_date_int(gameID)
    temp = df[(df['pitcherID']==playerID) & (df['gameDate']<gameDate)]
    stats = [playerID,gameID]
    trailing_abs = [100,300,500,1000,2000]
    for ab in trailing_abs:
        temp2 = temp.tail(ab)
        if len(temp) < 100:
            stats.extend([0,0,0,0,0,len(temp)])
            return stats
        else:
            stats.append(temp2['runs_created'].mean())
    stats.append(len(temp))
    return stats 

In [36]:
player_game_combos = []
for i in pitcher_game_df.itertuples():
    player_game_combos.append((i[1],i[2]))

In [39]:
l = []
start = time.time()
for i,item in enumerate(player_game_combos):
    l.append(get_stats(item))
    if i%100 == 0:
        print (time.time()-start)
        start = time.time()

0.04143881797790527
3.6726882457733154
3.68599796295166
3.7939300537109375
3.8074758052825928
3.988891839981079
4.030019044876099
3.8996241092681885
3.848785877227783
3.8613970279693604
3.743983745574951
3.8740909099578857
3.718600034713745
3.8266310691833496
3.9618239402770996
3.9005026817321777
3.9411771297454834
3.9117319583892822


KeyboardInterrupt: 

In [ ]:
p = Pool(processes=10)
# get_game_data is the function, game_list is a list (the argument)
relevant_info, error_output = p.map(get_game_data,game_list)